In [44]:
import os
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI"

# Prepare the data

# Prepare the Embedding Model

# Load data into Milvus


In [45]:
from pymilvus import MilvusClient
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

# Khởi tạo mô hình sentence-transformers
model = SentenceTransformer('all-MiniLM-L6-v2')
def emb_text(text):
    return model.encode(text).tolist()

# Đọc dữ liệu (giả định text_lines đã được định nghĩa)
from glob import glob
text_lines = []
for file_path in glob(r"D:\Big_project_2025\RAG_Milvus\data\*.md", recursive=True):
    with open(file_path, "r", encoding="utf-8") as file:
        file_text = file.read()
    text_lines += file_text.split("# ")
text_lines = [line.strip() for line in text_lines if line.strip() and line]

# Tạo embeddings
embeddings = [emb_text(line) for line in tqdm(text_lines, desc="Creating embeddings")]

# Cấu hình và chèn dữ liệu vào Milvus
milvus_client = MilvusClient(uri="http://localhost:19530")
collection_name = "my_rag_collection"

if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

milvus_client.create_collection(
    collection_name=collection_name,
    dimension=384,  # Phù hợp với sentence-transformers
    metric_type="IP",
    consistency_level="Bounded",
)

data = [{"id": i, "vector": vec, "text": text} for i, (text, vec) in enumerate(zip(text_lines, embeddings))]
milvus_client.insert(collection_name=collection_name, data=data)
print("Đã chèn xong!")

Creating embeddings: 100%|██████████| 7/7 [00:00<00:00, 15.55it/s]


Đã chèn xong!


# triển khai RAG

In [53]:
question = "thông tinh trường 256 Nguyễn Văn Cừ, An Hòa"

search_res = milvus_client.search(
    collection_name=collection_name,
    data=[emb_text(question)],
    limit=3,
    search_params={"metric_type": "IP", "params": {}},
    output_fields=["text"],
)

import json
retrieved_lines_with_distances = [(res["entity"]["text"], res["distance"]) for res in search_res[0]]
#print(json.dumps(retrieved_lines_with_distances, indent=4))
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)
print("Context:", context)

Context: THÔNG TIN TRƯỜNG
02923894050

phonghanhchinh@ctuet.edu.vn

02923894103

256 Nguyễn Văn Cừ, An Hòa, Ninh Kiều, Cần Thơ
Facebook: https://www.facebook.com/CTUT.CT/


###
SỨ MẠNG

       Đào tạo nguồn nhân lực có đạo đức tốt, có chuyên môn cao, có khả năng tiếp cận nghiên cứu khoa học, ứng dụng và chuyển giao công nghệ tiên tiến trong lĩnh vực kỹ thuật, công nghệ, đáp ứng nhu cầu phát triển của TP Cần Thơ, vùng ĐBSCL và cả nước.

###
TRIẾT LÝ GIÁO DỤC

         Đức trí - Kỹ năng - Sáng tạo - Hội nhập
